## Performance of the CPL
Author: Clara Hoffmann \
Measures the performance of the CPL on the precise validation set.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy as sp
from tqdm import tqdm
from scipy.integrate import simps, trapz
import matplotlib.pyplot as plt
import imageio
import multiprocessing
from helpers import (Fy, find_closest_element,  compute_coverage, #predict_single_density,
confidence_interval, confidence_interval, generate_fixed_terms, get_ci)
from density_predictor_class import density_predictor

In [2]:
# read in val data
#B_zeta = np.load('../../../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/Bzeta/B_zeta_val.npy')
B_zeta = np.load('../../../../data/commaai/extracted_coefficients/copula_cpl/Bzeta/B_zeta_val.npy')
B_zeta = B_zeta.reshape(B_zeta.shape[0], B_zeta.shape[2])
true_y = np.load('../../../../data/commaai/extracted_coefficients/copula_cpl/Bzeta/labels_val.npy')

density_path = '../../../../data/commaai/density/gaussian_density_filtered.csv'
density = pd.read_csv(density_path)

# number of points at which to evaluate the predictive densities
no_points = 500

In [3]:
# mc_dropout predictions:
preds = np.append(np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_2.npy'), 
                  np.load('../../../../data/commaai/predictions/mc_dropout/mc_preds_1.npy'), axis = 0)
preds_mc = np.mean(preds, axis = 1)

In [4]:
dens = density_predictor(B_zeta, true_y, density, no_points)

In [5]:
result_va_ridge = dens.get_density('va_ridge')
result_hmc_ridge = dens.get_density('hmc_ridge')
result_va_horse = dens.get_density('va_horseshoe')
result_hmc_horse = dens.get_density('hmc_horseshoe')

  1%|          | 61/10472 [00:00<00:17, 602.48it/s]

computing densities for each observation


  8%|▊         | 786/10472 [00:00<00:01, 7851.43it/s]

computing mean prediction for each observation


 12%|█▏        | 1287/10472 [00:00<00:00, 12860.87it/s]

computing variance prediction for each observation


100%|██████████| 10472/10472 [00:00<00:00, 12807.70it/s]


In [6]:
print('Validation Performance: ' +
      '\nMSE:' +
      '\n CPL VA/Ridge: ' + str(np.mean((true_y - np.array(result_va_ridge['mean predictions']))**2)) +
      '\n CPL HMC/Ridge: ' + str(np.mean((true_y - np.array(result_hmc_ridge['mean prediction']))**2)) +
      '\n MC-Dropout: ' + str(np.mean((true_y - preds_mc.reshape(10472,))**2)) +
      '\n CPL VA/Horseshoe: ' + str(np.mean((true_y - np.array(result_va_horse['mean prediction']))**2)) +
      '\nMAE:' +
      '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])))) +
      '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])))) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc.reshape(10472,)))) +
      '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])))) +
      '\nAccuracy:' +
      '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) <= 6)) +
       '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) <= 6)) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc.reshape(10472,)) <= 6)) +
      '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) <= 6)) +
      '\nAccuracy II :' +
      '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) <= 2)) +
      '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) <= 2)) +
     '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc.reshape(10472,)) <= 2)) +
    '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) <= 2)))

Validation Performance: 
MSE:
 CPL VA/Ridge: 6.280307985896987
 CPL HMC/Ridge: 6.280432798344231
 MC-Dropout: 6.3471034626787475
 CPL VA/Horseshoe: 5.655448433956707
MAE:
 CPL VA/Ridge: 1.340239464764066
 CPL HMC/Ridge: 1.3402254815961587
 MC-Dropout: 1.4046055209861767
 CPL VA/Horseshoe: 1.292931062885251
Accuracy:
 CPL VA/Ridge: 0.9808059587471352
 CPL HMC/Ridge: 0.9808059587471352
 MC-Dropout: 0.9836707410236822
 CPL VA/Horseshoe: 0.9821428571428571
Accuracy II :
 CPL VA/Ridge: 0.8220970206264324
 CPL HMC/Ridge: 0.8220015278838808
 MC-Dropout: 0.8008021390374331
 CPL VA/Horseshoe: 0.8273491214667685


In [7]:
print('Validation Performance: ' +
      '\nMSE:' +
      '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0)**2)) +
     '\n CPL VA/Ridge: ' + str(np.mean((true_y - np.array(result_va_ridge['mean predictions']))**2)) +
     '\n CPL HMC/Ridge: ' + str(np.mean((true_y - np.array(result_hmc_ridge['mean prediction']))**2)) +
     '\n CPL VA/Horseshoe: ' + str(np.mean((true_y - np.array(result_va_horse['mean prediction']))**2)) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean((true_y - np.array(result_hmc_horse['mean prediction']))**2)) +
      '\n MC-Dropout: ' + str(np.mean((true_y - preds_mc)**2)) +
      
    '\nMAE:' +
      '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0))) +
     '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])))) +
     '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])))) +
     '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])))) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])))) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc))) +
      
      '\nAccuracy:' +
     '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0) <= 6)) +
     '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) <= 6)) +
     '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) <= 6)) +
     '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) <= 6)) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])) <= 6)) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc) <= 6)) +
      
      '\nAccuracy II :' +
      '\n CPL Benchmark: ' + str(np.mean(np.abs(true_y - 0) <= 2)) +
     '\n CPL VA/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_va_ridge['mean predictions'])) <= 2)) +
     '\n CPL HMC/Ridge: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_ridge['mean prediction'])) <= 2)) +
     '\n CPL VA/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_va_horse['mean prediction'])) <= 2)) +
     '\n CPL HMC/Horseshoe: ' + str(np.mean(np.abs(true_y - np.array(result_hmc_horse['mean prediction'])) <= 2)) +
      '\n MC-Dropout: ' + str(np.mean(np.abs(true_y - preds_mc) < 2))
      
     )

Validation Performance: 
MSE:
 CPL Benchmark: 14.610926279602749
 CPL VA/Ridge: 6.280307985896987
 CPL HMC/Ridge: 6.280432798344231
 CPL VA/Horseshoe: 5.655448433956707
 CPL HMC/Horseshoe: 5.664642391219545
 MC-Dropout: 24.545156093878376
MAE:
 CPL Benchmark: 2.1570091673032854
 CPL VA/Ridge: 1.340239464764066
 CPL HMC/Ridge: 1.3402254815961587
 CPL VA/Horseshoe: 1.292931062885251
 CPL HMC/Horseshoe: 1.2928030458851116
 MC-Dropout: 3.4319043134426486
Accuracy:
 CPL Benchmark: 0.9321046600458365
 CPL VA/Ridge: 0.9808059587471352
 CPL HMC/Ridge: 0.9808059587471352
 CPL VA/Horseshoe: 0.9821428571428571
 CPL HMC/Horseshoe: 0.9821428571428571
 MC-Dropout: 0.8440418857139356
Accuracy II :
 CPL Benchmark: 0.6608097784568373
 CPL VA/Ridge: 0.8220970206264324
 CPL HMC/Ridge: 0.8220015278838808
 CPL VA/Horseshoe: 0.8273491214667685
 CPL HMC/Horseshoe: 0.8275401069518716
 MC-Dropout: 0.42038511442496296


In [ ]:
df = pd.DataFrame({'va_ridge' : result_va_ridge['densities'],
                 'hmc_ridge' : result_hmc_ridge['densities'],
                 'va_horse' : result_va_horse['densities'],
                 'hmc_horse' : result_hmc_horse['densities']})

In [ ]:
df.to_csv('../../../../data/commaai/predictions/cpl/densities.csv')